In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
X = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv",index_col=0)
X_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv", index_col=0)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

print(X.shape)
print(X_test.shape)

X_full = pd.concat([X,X_test],axis=0)

# print(X_full.shape)

# print(X_full.info())

n_col = [colname for colname in X_full.columns if X_full[colname].dtype in ["int64","float64"]]
cate_col = [colname for colname in X_full.columns if X_full[colname].dtype == "object"]
print(len(cate_col) + len(n_col))

imputer = SimpleImputer(strategy="mean")
imputer2 = SimpleImputer(strategy="most_frequent")
OH = OneHotEncoder(sparse=False,handle_unknown="ignore")

# print(X_full.shape)
X_full = pd.concat([pd.DataFrame(imputer.fit_transform(X_full[n_col]),index=X_full.index,columns=X_full[n_col].columns),X_full[cate_col]],axis=1)


#Delete columns with > 50% NaN values
X_full.drop(["PoolQC","MiscFeature","Alley","Fence","FireplaceQu"],inplace=True,axis=1)

#Get Category columns
cate_col = [colname for colname in X_full.columns if X_full[colname].dtype == "object"]

X_full = pd.concat([pd.DataFrame(imputer2.fit_transform(X_full[cate_col]),index=X_full.index,columns=X_full[cate_col].columns),X_full[n_col]],axis=1)

OH_col = pd.DataFrame(OH.fit_transform(X_full[cate_col]),index=X_full.index)

X_full = pd.concat([X_full,OH_col],axis=1)

X_full.drop(cate_col,inplace=True,axis=1)

print(X_full.isnull().sum().sort_values(ascending=False))
print(X_full.shape)

X_clean = X_full.iloc[:1460]
X_test_clean = X_full.iloc[1460:]
y = X_clean["SalePrice"]
X_clean.drop("SalePrice",axis=1,inplace=True)
X_test_clean.drop("SalePrice",axis=1,inplace=True)

print(X_clean.shape)
print(X_test_clean.shape)


(1460, 80)
(1459, 79)
80
233           0
47            0
61            0
60            0
59            0
             ..
137           0
136           0
135           0
134           0
MSSubClass    0
Length: 271, dtype: int64
(2919, 271)
(1460, 270)
(1459, 270)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


XGBRegressor Test

In [3]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

X_train, X_valid, y_train, y_valid = train_test_split(X_clean,y,test_size=.2,random_state=1,train_size=.8)

my_model_2 = XGBRegressor(n_estimators=500,random_state=0) # Your code here

# Fit the model
my_model_2.fit(X_train,y_train, early_stopping_rounds=10,eval_set=[(X_valid,y_valid)],verbose=False) # Your code here

# Get predictions
predictions_2 = my_model_2.predict(X_valid) # Your code here

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2,y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

pred_final = my_model_2.predict(X_test_clean)

Mean Absolute Error: 16462.55486140839


In [4]:
# Save test predictions to file

output = pd.DataFrame({'Id': X_test_clean.index,
                       'SalePrice': pred_final})
output.to_csv('submission.csv', index=False)

print("Done!!!")

Done!!!
